In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
#df = pd.read_csv('ECE_199_Complaints_sample_1.csv')
#stop_words = np.genfromtxt('Stop Words listing.txt', dtype='str').tolist()

## Data Preprocessing

In [2]:
df = pd.read_csv('Complaints.csv')
df_recall = pd.read_csv('Recalls.csv')
df.shape
df.columns

C:\Users\Xiaoxin Xu\AppData\Local\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Complaint ID', 'NHTSA Complaint Reference Number', 'Manufacturer Name',
       'Vehicle Equipment Make', 'Vehicle Equipment Model', 'Model Year',
       'Date Of Incident', 'Component Description', 'Component System',
       'Component Subsystem', 'Consumer City', 'Consumer State', 'VIN Number',
       'Date Added', 'Date Received', 'Vehicle Mileage',
       'Complaint Description', 'Complaint Source', 'Police Report',
       'Purchase Date', 'Vehicle Speed', 'Type Of Tire Failure',
       'Manufacture Date', 'Product Type', 'Vehicle Age', 'Clean Manufacturer',
       'Clean Make', 'Clean Model', 'Model Series', 'Vehicle Segment',
       'Model Sales', 'Series Sales', 'Record Quarter', 'Record Year Month',
       'Vehicle Kind', 'Vehicle Kind Component',
       'Vehicle Kind Full Component'],
      dtype='object')

In [3]:
df['Complaint Description'] = df['Complaint Description'].astype(str)
df['Complaint Description'] = df['Complaint Description'].str.upper()
df['Complaint ID'] = df['Complaint ID'].astype(str)

## Extract Phrases

In [5]:
def extract_phrases(complaint):
    """
    Return a list of unique phrases in one complaint
    """
    # get sentences
    sen = re.sub(r',|\.|\:|\;|\?|\!',' ,', complaint) 
    
    #remove digits
    sen = ' '.join(' ' if any(c.isdigit() for c in s) else s for s in sen.split()) 
    s_list = re.split(r',|\.|\:|\;|\?|\!', sen)
    temp = []
    
    #get phrases
    for s in s_list:
        s = re.sub(r'[^\w\s]',' ', s)
        words = s.split()
        for i in range(5):
            for j in range(len(words) - i):
                temp.append(' '.join(words[j:j + i + 1]))
    
    #with no duplicates
    return list(set(temp))            

In [6]:
def phrase_table(df):
    '''
    Return table with phrases and corresponding complaint ids
    '''
    phrases = []
    cids = []

    for i in range(len(df)):
        cid = df['Complaint ID'][i]

        temp_phrases = extract_phrases(df['Complaint Description'][i])  #extract
        temp_ids = [cid] * len(temp_phrases)
        phrases = phrases + temp_phrases
        cids = cids + temp_ids
    
    d = pd.DataFrame({'Phrase': phrases})
    d['Complaint ID'] = cids
    return d

In [7]:
d100 = phrase_table(df.head(100))

## Classify Recall Class

In [12]:
df_recall = pd.read_csv('Recalls.csv')
recall_camp = list(df_recall['Recall Campaign Number'].astype(str))
vehicle_kind = list(df_recall['Vehicle Kind'].astype(str))
vehicle_kind_component = list(df_recall['Vehicle Kind Component'].astype(str))

In [14]:
df_verified = pd.DataFrame({'Complaint': df['Complaint Description']})
df_verified['Complaint ID'] = df['Complaint ID'] 
df_verified.shape

(1249653, 2)

In [16]:
for i in range(len(df_verified)):
    rcn = re.findall('\d\d[CEITVX]\d{3,6}', df['Complaint Description'][i])
    if rcn:
        df_verified.set_value(i, 'Recall Class', 'Campaign Mentioned')
        for j in rcn:
            if j in recall_camp:
                df_verified.set_value(i, 'Recall Class', 'Campaign Verified')
                break
    
    elif df['Vehicle Kind'][i] in vehicle_kind:
        if df['Vehicle Kind Component'][i] in vehicle_kind_component:
            df_verified.set_value(i, 'Recall Class', 'Recall Kind Associated')
        else:
            df_verified.set_value(i, 'Recall Class', 'Recall Kind Associated but not Component')        
    else:
        df_verified.set_value(i, 'Recall Class', 'Not Recall Matched')

In [17]:
dencode = {'Campaign Verified': 1, 'Campaign Mentioned':2, 'Recall Kind Associated':3, 
           'Recall Kind Associated but not Component':4, 'Not Recall Matched':5}

In [32]:
df_verified['Class'] = df_verified['Recall Class'].apply(lambda x: dencode[x])
df_verified['Recall'] = df_verified['Class'].apply(lambda x: 1 if x < 5 else 0)
df_verified.head()

C:\Users\Xiaoxin Xu\AppData\Local\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Complaint,Complaint ID,Recall Class,Class,Recall
0,SEAT JAMS DUE TO FLOOR FLEXING. THE PROBLEM WA...,268713,Recall Kind Associated but not Component,4,1
1,SEAT JAMS DUE TO FLOOR FLEXING. THE PROBLEM WA...,268718,Recall Kind Associated but not Component,4,1
2,RIGHT REAR LIGHTING SYSTEM TO INCLUDE TURN SIG...,717436,Recall Kind Associated,3,1
3,RIGHT REAR LIGHTING SYSTEM TO INCLUDE TURN SIG...,717437,Recall Kind Associated but not Component,4,1
4,THE HARNESS SNAPPED WHICH CAUSED THE CHILD TO ...,402240,Not Recall Matched,5,0


In [198]:
df_verified.to_csv('Complaints_Verified.csv')